In [145]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
import pandas as pd
import numpy as np
import random


df = pd.read_csv('CreditPrediction.csv')
df = df.drop(columns=['Unnamed: 19', 'CLIENTNUM'])
print(len(df))

df = df.drop_duplicates(ignore_index=True)
df


10167


,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45.0,M,3,High School,Married,$60K - $80K,Blue,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061
1,49.0,F,5,Graduate,NaN,Less than $40K,Blue,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105
2,51.0,M,3,Graduate,Married,$80K - $120K,Blue,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000
3,40.0,F,4,High School,NaN,Less than $40K,Blue,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760
4,40.0,M,3,Uneducated,Married,$60K - $80K,NaN,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10127,37.0,F,3,High School,NaN,Unknown,NaN,18.0,4.0,1,3,2179.0,0,0.725,2568,48,0.297,0.000
10128,53.0,M,2,College,NaN,$40K - $60K,Blue,33.0,3.0,3,1,4065.0,2180,0.321,1271,36,0.200,0.536
10129,56.0,F,2,Uneducated,Married,Unknown,Blue,43.0,6.0,3,2,3068.0,1765,0.853,4611,73,0.698,0.575
10130,37.0,F,2,Graduate,Married,Less than $40K,NaN,32.0,6.0,1,3,1438.3,0,0.707,4733,89,0.816,0.000


In [146]:
df = df[df['Customer_Age'] <= 100]
print(len(df))

10115


In [147]:
X = df
print(set(X['Gender']))

{'F', nan, 'M'}


In [148]:
X.loc[:, 'Gender'] = X['Gender'].replace({'M': 1, 'F': 0})

X.head()

,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45.0,1.0,3,High School,Married,$60K - $80K,Blue,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061
1,49.0,0.0,5,Graduate,NaN,Less than $40K,Blue,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105
2,51.0,1.0,3,Graduate,Married,$80K - $120K,Blue,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000
3,40.0,0.0,4,High School,NaN,Less than $40K,Blue,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760
4,40.0,1.0,3,Uneducated,Married,$60K - $80K,NaN,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000


In [149]:
X['Education_Level'].isna().sum()

0

In [150]:
# X = pd.get_dummies(X, columns=['Education_Level'])
X = X.drop(columns=['Education_Level'])

X.head()

,Customer_Age,Gender,Dependent_count,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45.0,1.0,3,Married,$60K - $80K,Blue,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061
1,49.0,0.0,5,NaN,Less than $40K,Blue,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105
2,51.0,1.0,3,Married,$80K - $120K,Blue,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000
3,40.0,0.0,4,NaN,Less than $40K,Blue,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760
4,40.0,1.0,3,Married,$60K - $80K,NaN,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000


In [151]:
print(set(X['Marital_Status']))

{nan, 'Divorced', 'Unknown', 'Single', 'Married'}


In [152]:
# X['Marital_Status'].fillna(random.choice(['Single', 'Unknown', 'Divorced', 'Married']), inplace=True)
# X.head()

In [153]:
# X = pd.get_dummies(X, columns=['Marital_Status'])
X = X.drop(columns=['Marital_Status'])

X.head()

,Customer_Age,Gender,Dependent_count,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45.0,1.0,3,$60K - $80K,Blue,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061
1,49.0,0.0,5,Less than $40K,Blue,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105
2,51.0,1.0,3,$80K - $120K,Blue,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000
3,40.0,0.0,4,Less than $40K,Blue,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760
4,40.0,1.0,3,$60K - $80K,NaN,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000


In [154]:
print(set(X['Income_Category']))

{'$120K +', 'Less than $40K', 'Unknown', '$40K - $60K', '$80K - $120K', '$60K - $80K'}


In [155]:
income_order = {
    'Less than $40K': 0,
    '$40K - $60K': 1,
    '$60K - $80K': 2,
    '$80K - $120K': 3,
    '$120K +': 4,
    'Unknown': 5
}

In [156]:
X['Income_Category'] = X['Income_Category'].map(income_order)
X.head()

,Customer_Age,Gender,Dependent_count,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45.0,1.0,3,2,Blue,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061
1,49.0,0.0,5,0,Blue,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105
2,51.0,1.0,3,3,Blue,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000
3,40.0,0.0,4,0,Blue,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760
4,40.0,1.0,3,2,NaN,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000


In [157]:
X['Card_Category'].isna().sum()

1912

In [158]:
print(set(X['Card_Category']))

{'Blue', nan, 'Gold', 'Silver', 'Platinum'}


In [159]:
# X = pd.get_dummies(X, columns=['Card_Category'])

card_category_map = {
    'Blue': 0,
    'Silver':1,
    'Gold': 2,
    'Platinum': 3
}

X['Card_Category'] = X['Card_Category'].map(card_category_map)

In [160]:
X.head()

,Customer_Age,Gender,Dependent_count,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45.0,1.0,3,2,0.0,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061
1,49.0,0.0,5,0,0.0,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105
2,51.0,1.0,3,3,0.0,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000
3,40.0,0.0,4,0,0.0,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760
4,40.0,1.0,3,2,NaN,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000


In [161]:
def fill_na_with_mode(df, columns):
    for column in columns:
        if column in df.columns:
            max_value = df[column].mode()
            df[column].fillna(max_value, inplace=True)
    return df

def fill_na_with_mean(df, columns):
    for column in columns:
        if column in df.columns:
            max_value = df[column].mean()
            df[column].fillna(max_value, inplace=True)
    return df

In [162]:
def report_na(df):
    na_counts = df.isna().sum()
    total_na = df.isna().sum().sum()
    
    print("Number of NaN values in each column:")
    print(na_counts)
    print("\nTotal number of NaN values in the DataFrame:", total_na)

    return na_counts, total_na

report_na(X)

Number of NaN values in each column:
Customer_Age                   0
Gender                       199
Dependent_count                0
Income_Category                0
Card_Category               1912
Months_on_book               219
Total_Relationship_Count      20
Months_Inactive_12_mon         0
Contacts_Count_12_mon          0
Credit_Limit                   0
Total_Revolving_Bal            0
Total_Amt_Chng_Q4_Q1           0
Total_Trans_Amt                0
Total_Trans_Ct                 0
Total_Ct_Chng_Q4_Q1            0
Avg_Utilization_Ratio          0
dtype: int64

Total number of NaN values in the DataFrame: 2350


(Customer_Age                   0
 Gender                       199
 Dependent_count                0
 Income_Category                0
 Card_Category               1912
 Months_on_book               219
 Total_Relationship_Count      20
 Months_Inactive_12_mon         0
 Contacts_Count_12_mon          0
 Credit_Limit                   0
 Total_Revolving_Bal            0
 Total_Amt_Chng_Q4_Q1           0
 Total_Trans_Amt                0
 Total_Trans_Ct                 0
 Total_Ct_Chng_Q4_Q1            0
 Avg_Utilization_Ratio          0
 dtype: int64,
 2350)

In [163]:
X = fill_na_with_mode(X, ['Dependent_count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon','Total_Revolving_Bal', 'Total_Amt_Chng_Q4_Q1', 'Avg_Utilization_Ratio'])
X = fill_na_with_mean(X, ['Card_Category', 'Months_on_book', 'Total_Relationship_Count' , 'Gender'])
report_na(X)

Number of NaN values in each column:
Customer_Age                0
Gender                      0
Dependent_count             0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
dtype: int64

Total number of NaN values in the DataFrame: 0


(Customer_Age                0
 Gender                      0
 Dependent_count             0
 Income_Category             0
 Card_Category               0
 Months_on_book              0
 Total_Relationship_Count    0
 Months_Inactive_12_mon      0
 Contacts_Count_12_mon       0
 Credit_Limit                0
 Total_Revolving_Bal         0
 Total_Amt_Chng_Q4_Q1        0
 Total_Trans_Amt             0
 Total_Trans_Ct              0
 Total_Ct_Chng_Q4_Q1         0
 Avg_Utilization_Ratio       0
 dtype: int64,
 0)

In [164]:
X

,Customer_Age,Gender,Dependent_count,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,45.0,1.0,3,2,0.000000,39.0,5.0,1,3,12691.0,777,1.335,1144,42,1.625,0.061
1,49.0,0.0,5,0,0.000000,44.0,6.0,1,2,8256.0,864,1.541,1291,33,3.714,0.105
2,51.0,1.0,3,3,0.000000,36.0,4.0,1,0,3418.0,0,2.594,1887,20,2.333,0.000
3,40.0,0.0,4,0,0.000000,34.0,3.0,4,1,3313.0,2517,1.405,1171,20,2.333,0.760
4,40.0,1.0,3,2,0.085335,21.0,5.0,1,0,4716.0,0,2.175,816,28,2.500,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10127,37.0,0.0,3,5,0.085335,18.0,4.0,1,3,2179.0,0,0.725,2568,48,0.297,0.000
10128,53.0,1.0,2,1,0.000000,33.0,3.0,3,1,4065.0,2180,0.321,1271,36,0.200,0.536
10129,56.0,0.0,2,5,0.000000,43.0,6.0,3,2,3068.0,1765,0.853,4611,73,0.698,0.575
10130,37.0,0.0,2,0,0.085335,32.0,6.0,1,3,1438.3,0,0.707,4733,89,0.816,0.000


In [165]:
def apply_iqr_filtering(df):
    for feature in df.columns:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]
    return df

X = apply_iqr_filtering(X)
X

,Customer_Age,Gender,Dependent_count,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
10,42.0,1.0,5,4,0.0,31.0,5.0,3,2,6748.0,1467,0.831,1201,42,0.680,0.217
14,57.0,0.0,2,0,0.0,48.0,5.0,2,2,2436.0,680,1.190,1570,29,0.611,0.279
19,45.0,0.0,2,5,0.0,37.0,6.0,1,2,14470.0,1157,0.966,1207,21,0.909,0.080
21,62.0,0.0,0,0,0.0,49.0,2.0,3,3,1438.3,0,1.047,692,16,0.600,0.000
23,47.0,0.0,4,0,0.0,36.0,3.0,3,2,2492.0,1560,0.573,1126,23,0.353,0.626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10048,48.0,1.0,4,3,0.0,44.0,2.0,3,3,9524.0,1449,0.559,7527,58,0.657,0.152
10067,49.0,0.0,4,1,0.0,36.0,5.0,2,3,4167.0,0,0.581,7590,87,0.776,0.000
10125,30.0,1.0,2,1,0.0,36.0,4.0,3,3,5281.0,0,0.535,8395,62,0.722,0.000
10129,56.0,0.0,2,5,0.0,43.0,6.0,3,2,3068.0,1765,0.853,4611,73,0.698,0.575


In [166]:
Y = X['Credit_Limit']
X = X.drop(columns=['Credit_Limit'])



In [167]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)
scaler = StandardScaler()

# Fit and transform the training data
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [168]:
# Train the RandomForestRegressor model
model = RandomForestRegressor(max_depth=15, min_samples_split=10, n_estimators=85, random_state=25)
model.fit(X_train_transformed, y_train)

# Predict and evaluate the model
Y_pred = model.predict(X_test_transformed)

# Calculate metrics
mse = mean_squared_error(y_test, Y_pred)
r2 = r2_score(y_test, Y_pred)

# Print results
print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 2985906.203998545
R2 Score: 0.8270731374157205
